In [12]:
import json
import cv2
import glob
import os
import pandas as pd
from pandas import json_normalize
import numpy as np

In [5]:
train_imgs = glob.glob('./train2014/*')
train_masks = glob.glob('./train_mask/*')
val_imgs = glob.glob('./val2014/*')
val_masks = glob.glob('./val_mask/*')

In [7]:
print("train_imgs", len(train_imgs))
print("train_masks", len(train_masks)) 
print("val_imgs", len(val_imgs)) 
print("val_masks", len(val_masks))

# 삭제 후 개수 동일 train -> 26437, val -> 1508

train_imgs 26437
train_masks 26437
val_imgs 1508
val_masks 1508


In [9]:
ext_check = set()
for imgs in train_imgs:
    root, ext = os.path.splitext(imgs)
    ext_check.add(ext)
print("train_imgs", ext_check)

ext_check = set()
for imgs in train_masks:
    root, ext = os.path.splitext(imgs)
    ext_check.add(ext)
print("train_masks", ext_check)

ext_check = set()
for imgs in val_imgs:
    root, ext = os.path.splitext(imgs)
    ext_check.add(ext)
print("val_imgs", ext_check)

ext_check = set()
for imgs in val_masks:
    root, ext = os.path.splitext(imgs)
    ext_check.add(ext)
print("val_masks", ext_check)

# train img는 jpg로 mask img는 png로 구성되어 있음

train_imgs {'.jpg'}
train_masks {'.png'}
val_imgs {'.jpg'}
val_masks {'.png'}


In [10]:
# train json 확인
with open('densepose_coco_2014_train.json', 'r') as f:
    json_data = json.load(f)

In [14]:
# 카테고리 확인
df_categories = json_normalize(json_data['categories'])
print(df_categories) # keypoints와 더불어 skeleton 정보가 있음

  supercategory  id    name  \
0        person   1  person   

                                           keypoints  \
0  [nose, left_eye, right_eye, left_ear, right_ea...   

                                            skeleton  
0  [[16, 14], [14, 12], [17, 15], [15, 13], [12, ...  


In [32]:
keypoints_idx = dict()
for idx, key, in enumerate(json_data['categories'][0]['keypoints']):
    print(idx,':', key)
    keypoints_idx[idx] = key
    
# 얼굴에 해당되는 부위는 필요 없음 0 ~ 4 필요 없는 부분
print("")
for a, b in json_data['categories'][0]['skeleton']:
    print(keypoints_idx[a-1], "-", keypoints_idx[b-1])

0 : nose
1 : left_eye
2 : right_eye
3 : left_ear
4 : right_ear
5 : left_shoulder
6 : right_shoulder
7 : left_elbow
8 : right_elbow
9 : left_wrist
10 : right_wrist
11 : left_hip
12 : right_hip
13 : left_knee
14 : right_knee
15 : left_ankle
16 : right_ankle

left_ankle - left_knee
left_knee - left_hip
right_ankle - right_knee
right_knee - right_hip
left_hip - right_hip
left_shoulder - left_hip
right_shoulder - right_hip
left_shoulder - right_shoulder
left_shoulder - left_elbow
right_shoulder - right_elbow
left_elbow - left_wrist
right_elbow - right_wrist
left_eye - right_eye
nose - left_eye
nose - right_eye
left_eye - left_ear
right_eye - right_ear
left_ear - left_shoulder
right_ear - right_shoulder


In [25]:
alchera_string = "배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14".split('/')
for i in alchera_string:
    print(i.strip())

배경 - 0
몸통 - 1
오른손 - 2
왼손 - 3
왼발 - 4
오른발 - 5
오른쪽 허벅지 - 6
왼쪽 허벅지 - 7
오른쪽 종아리 - 8
왼쪽 종아리 - 9
왼쪽 팔 - 10
오른쪽 팔 - 11
왼쪽 전완 - 12
오른쪽 전완 - 13
머리 - 14


In [41]:
train_img_check = json_data['images']
print(len(train_img_check))

26437


In [67]:
size_dict = set()
height_list = list()
width_list = list()
for img in train_img_check:
    size_dict.add(str(img['height']) + 'x' + str(img['width']))
    height_list.append(int(img['height']))
    width_list.append(int(img['width']))
print(size_dict)
print(len(size_dict))
print('maximum height, width : ', max(height_list), max(width_list))
print('minimum height, width : ', min(height_list), min(width_list))
print('average height, width : ', sum(height_list) / len(height_list), sum(width_list) / len(width_list))

{'358x477', '480x507', '640x448', '572x640', '640x580', '640x522', '640x296', '640x368', '628x640', '640x426', '262x350', '523x640', '519x640', '531x640', '298x448', '640x529', '405x640', '328x500', '363x640', '363x500', '456x640', '604x454', '478x500', '308x385', '347x500', '558x640', '587x640', '640x466', '331x640', '281x500', '640x355', '480x312', '470x314', '640x409', '506x640', '361x500', '414x500', '350x640', '600x631', '422x563', '334x500', '576x576', '370x640', '277x370', '305x500', '339x418', '277x500', '640x414', '542x480', '422x640', '470x340', '500x500', '259x640', '640x614', '640x333', '175x500', '461x640', '406x610', '500x442', '497x500', '640x589', '392x425', '488x500', '640x358', '576x396', '500x298', '340x510', '315x640', '500x401', '640x603', '640x427', '379x500', '500x479', '368x500', '500x377', '500x325', '412x640', '612x640', '391x640', '640x433', '640x523', '640x588', '455x500', '395x640', '399x600', '389x640', '640x621', '286x320', '492x500', '640x361', '500x347'

In [60]:
train_anno_check = json_data['annotations']
print(len(train_anno_check))

100403


In [62]:
# annotation의 경우 visualization 코드를 분석해보면 dp_mask만을 이용하는 것 같음

In [63]:
# val json 확인
with open('densepose_coco_2014_minival.json', 'r') as f:
    json_data = json.load(f)

In [64]:
val_img_check = json_data['images']
print(len(val_img_check))

1508


In [68]:
size_dict = set()
height_list = list()
width_list = list()
for img in val_img_check:
    size_dict.add(str(img['height']) + 'x' + str(img['width']))
    height_list.append(int(img['height']))
    width_list.append(int(img['width']))
print(size_dict)
print(len(size_dict))
print('maximum height, width : ', max(height_list), max(width_list))
print('minimum height, width : ', min(height_list), min(width_list))
print('average height, width : ', sum(height_list) / len(height_list), sum(width_list) / len(width_list))

{'639x640', '640x334', '428x640', '263x640', '358x500', '640x425', '447x640', '605x640', '640x287', '393x640', '436x640', '640x488', '640x580', '329x500', '640x444', '520x369', '408x640', '401x640', '539x640', '470x640', '621x448', '432x640', '427x640', '392x640', '512x640', '640x561', '500x375', '640x417', '640x512', '612x612', '396x576', '640x426', '438x500', '640x508', '640x593', '500x424', '500x409', '588x640', '500x468', '531x640', '505x640', '500x332', '640x428', '640x443', '500x336', '425x640', '378x500', '312x640', '640x635', '420x640', '489x640', '478x640', '499x640', '640x483', '640x484', '488x640', '640x392', '344x500', '396x640', '640x246', '363x640', '433x640', '640x329', '466x640', '299x500', '640x536', '640x568', '444x640', '455x640', '640x481', '500x337', '333x500', '454x640', '500x335', '633x640', '449x640', '406x640', '640x439', '453x640', '371x640', '500x368', '368x640', '356x500', '558x640', '640x421', '378x640', '500x276', '640x395', '320x640', '559x640', '281x500'

In [69]:
val_anno_check = json_data['annotations']
print(len(val_anno_check))

5673


In [71]:
# real_test_dataset

In [73]:
test_set = glob.glob("../test_dataset/*")

In [74]:
ext_check = set()
for imgs in test_set:
    root, ext = os.path.splitext(imgs)
    ext_check.add(ext)
print("test_imgs", ext_check) # png와 jpg있음
# jpg과 원본 png는 마스크 이미지

test_imgs {'.png', '.jpg'}


In [75]:
test_set_origin = glob.glob("../test_dataset/*.jpg")
test_set_mask = glob.glob("../test_dataset/*.png")

In [76]:
print(len(test_set_origin))
print(len(test_set_mask)) # 400장 존재

400
400


In [82]:
from PIL import Image

img_info = dict(heights=[], widths=[], means=[], stds=[])
for path in test_set_origin:
    img = np.array(Image.open(path))
    h, w, _ = img.shape
    img_info['heights'].append(h)
    img_info['widths'].append(w)
    img_info['means'].append(img.mean(axis=(0,1)))
    img_info['stds'].append(img.std(axis=(0,1)))

In [83]:
# 실제 데이터셋을 통한 확인
print(f'Minimum height for dataset is {np.min(img_info["heights"])}')
print(f'Maximum height for dataset is {np.max(img_info["heights"])}')
print(f'Average height for dataset is {int(np.mean(img_info["heights"]))}')
print(f'Minimum width for dataset is {np.min(img_info["widths"])}')
print(f'Maximum width for dataset is {np.max(img_info["widths"])}')
print(f'Average width for dataset is {int(np.mean(img_info["widths"]))}')

# Normalizae에 사용할 것
print(f'RGB Mean: {np.mean(img_info["means"], axis=0) / 255.}')
print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0) / 255.}')

Minimum height for dataset is 505
Maximum height for dataset is 1920
Average height for dataset is 1661
Minimum width for dataset is 811
Maximum width for dataset is 3264
Average width for dataset is 1268
RGB Mean: [0.51432493 0.49195858 0.46158501]
RGB Standard Deviation: [0.2402324  0.22889128 0.22452611]


In [84]:
only_one_img = test_set_mask[0]

In [89]:
img = np.array(Image.open(only_one_img))
print(np.unique(img))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
